In [ ]:
import numpy as np
import pandas as pd
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.optimize import minimize

from libfunc import compute_log_likelihood

from scipy.stats import binom

import mle
import flmp

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# sim data
N_SAMPLES = 100

In [ ]:
# params = {
#     "flmp": np.random.uniform(low=-5, high=5, size=(N, 2, K)),
#     "mle": np.random.uniform(low=-5, high=5, size=(N, 4))
# }
samples = {
    "FLMP": np.zeros((N,M,K)),
    "MLE": np.zeros((N,M,K))
}
for i, d in enumerate(data):
    _, c_A, c_V, std_A, std_V = mle.fit(np.ones(4), d, n_samples)
    _, tA, tV = flmp.fit(np.ones(K*2), d, n_samples, K)

    p_A, p_V, p_AV = flmp.compute_probs(tA, tV)
    samples["FLMP"][i] = binom.rvs(N_SAMPLES, np.vstack([p_A.T, p_V.T, p_AV]))
    
    p_A, p_V, p_AV = mle.compute_probs(c_A, c_V, np.exp(std_A), np.exp(std_V))
    samples["MLE"][i]  = binom.rvs(N_SAMPLES, np.vstack([p_A.T, p_V.T, p_AV]))

In [ ]:
tmp = {
        "Fitting Model": [],
        "Sampling Model": [], 
        "Subject": [],
        "NLL": []
}
for model in samples:
    for i, Subject in enumerate(samples[model]):
        nll, _, _, _, _ = mle.fit(np.ones(4), Subject, N_SAMPLES)
        tmp["Fitting Model"].append("MLE")
        tmp["Sampling Model"].append(model)
        tmp["Subject"].append(f"{i+1}")
        tmp["NLL"].append(nll)
        

        nll, _, _ = flmp.fit(np.ones(K*2), Subject, N_SAMPLES, K)
        tmp["Fitting Model"].append("FLMP")
        tmp["Sampling Model"].append(model)
        tmp["Subject"].append(f"{i+1}")
        tmp["NLL"].append(nll)

In [ ]:
pd.DataFrame.from_dict(tmp).set_index(["Fitting Model", "Subject"])

In [ ]:
df.loc["a", "b"] = 1

In [ ]:
df